# Quantum ML Test
Requirements:

Qiskit installed (pip install qiskit qiskit-machine-learning pandas scikit-learn)

An IBM Quantum account (get your API token from IBM Quantum)

## 1. Load and Preprocess the Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('test/diabetes.csv')

# Features and labels
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# For quantum, reduce to 2 features (for visualization and circuit size)
X_small = X_scaled[:, :2]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_small, y, test_size=0.2, random_state=42, stratify=y)

## 2. Set Up Qiskit and IBM Quantum Account

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save your IBM Quantum API token in your environment or paste it here
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_IBM_QUANTUM_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)
print("Using backend:", backend)

## 3. Build and Train the Quantum SVM

In [ ]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import algorithm_globals
from sklearn.svm import SVC

algorithm_globals.random_seed = 42

# Feature map for 2 features
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)

# Quantum kernel using the real backend
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

# Compute the kernel matrices
kernel_train = quantum_kernel.evaluate(x_vec=X_train)
kernel_test = quantum_kernel.evaluate(x_vec=X_test, y_vec=X_train)

# Train SVM with precomputed quantum kernel
qsvc = SVC(kernel='precomputed')
qsvc.fit(kernel_train, y_train)

# Predict and evaluate
y_pred = qsvc.predict(kernel_test)

from sklearn.metrics import accuracy_score, classification_report
print("Test accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

## 4. Notes

- Quantum resources are limited: For a real backend, jobs may queue and take time. For quick tests, use backend = service.get_backend('ibmq_qasm_simulator').
- Feature reduction: Quantum SVMs are currently practical for 2-3 features due to circuit depth and noise. For more features, use PCA or select the most relevant ones.
- API Token: Replace "YOUR_IBM_QUANTUM_TOKEN" with your actual token if not already saved.

## Full Example

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import algorithm_globals
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 1. Load and preprocess data
df = pd.read_csv('test/diabetes.csv')
X = df.drop('Outcome', axis=1)
y = df['Outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_small = X_scaled[:, :2]
X_train, X_test, y_train, y_test = train_test_split(X_small, y, test_size=0.2, random_state=42, stratify=y)

# 2. Set up IBM Quantum backend
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)
print("Using backend:", backend)

# 3. Quantum SVM
algorithm_globals.random_seed = 42
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)
kernel_train = quantum_kernel.evaluate(x_vec=X_train)
kernel_test = quantum_kernel.evaluate(x_vec=X_test, y_vec=X_train)
qsvc = SVC(kernel='precomputed')
qsvc.fit(kernel_train, y_train)
y_pred = qsvc.predict(kernel_test)

# 4. Results
print("Test accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))